In [1]:
0.9 ** 5

0.5904900000000001

In [1]:
# from pyspark.sql import SparkSession
# import os
# os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6"
# spark = SparkSession.builder.appName("tkt_kwd_model").enableHiveSupport().getOrCreate()
# train_sql_query = ""
# spark_df = spark.sql(sql_query)
# data = spark_df.toPandas()

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
def get_model_input(data, all_feature_list, target='isorder', over_sample_rate=-1, task='train',
                    item_col='viewspotid_enc_index', seq_col='pkgid_enc_index_click_pkgid_list_u', 
                    seq_len=10, item_enc_max=100000):
    df = data[[feat['feat_name'] for feat in all_feature_list] + [item_col, seq_col]]
    target = data[target]
    y = target.values.astype('int32')
    if over_sample_rate > 0:
        pos = df[y == 1]
        neg = df[y == 0]
        X = pd.concat([pos for _ in range(over_sample_rate)] + [neg], axis=0)
        y = np.array([1] * (len(pos) * over_sample_rate) + [0] * len(neg))
    else:
        X = df
    if task == 'train': # shuffle the dataset when training
        idx = [i for i in range(len(y))]
        random.shuffle(idx)
        X = X.iloc[idx]
        y = y[idx]
    seq_input = X[seq_col]
    item_input = X[item_col]
    X = X.drop([seq_col, item_col], axis=1).values.astype('float32')
    seq_input = seq_input.fillna('0').str.split(',', n=seq_len, expand=True).iloc[:, :seq_len].fillna(0).values.astype('int32')
    seq_input = np.where(seq_input <= item_enc_max, seq_input, 0)
    # tf.keras.preprocessing.sequence.pad_sequences(ttt, padding='post', maxlen=2)
    item_input = item_input.values.astype('int32')
    return [X[:, i] for i in range(X.shape[1])], y, seq_input.reshape(seq_input.shape[0], seq_input.shape[1], 1), item_input.reshape(-1, 1)

In [3]:
import sys
import os 
sys.path.append("..")
sys.path.append(".")
import random
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout
from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC, Accuracy, BinaryAccuracy
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()
from WDL.model import WideDeep
from WDL.model_with_seq import WideDeepDIN
from WDL.modules import sparseFeature, denseFeature

In [4]:
from utils.get_metrics import get_topn, get_gauc# , get_topn_v2, get_gauc_v2

In [5]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [6]:
# sample = -1
# train_file = '../dataset/tkt_kwd_train_721_727.csv'
# data = pd.read_csv(train_file)
# print('train samples:', len(data))
# if sample == -1:
#     data = pd.read_csv(file)
# else:
#     data = pd.read_csv(file, iterator=True)
#     data = data.get_chunk(sample)

## 特征

In [7]:
# wide_dense_features = [x.strip() for x in 'around_tour, isholiday, hot_p1m, ctr_p1m, order_p1m, gmv_p1m, unitgmv_p1m, comment_p, comment_pl2m, hot_pla1m, ctr_pla1m, order_pla1m, gmv_pla1m, unitgmv_pla1m, queryctr_pldk1m, new_p1m, season_p, basic_p, kwdcoeff_p, text_score, local_poi_distance, local_district_distance, poi_district_distance, businessstatus, dat, date_type, holiday_type, day_of_week, week_of_year, month, isheat, imquality_score, ishighqualitypoi, iscoverimage, hasticket, inchina, isfree'.split(',')]
# deep_dense_features = [x.strip() for x in 'order_cnt_3m_p, order_cnt_1m_p, order_cnt_15d_p, order_cnt_7d_p, order_cnt_3d_p, gmv_3m_p, gmv_1m_p, gmv_15d_p, gmv_7d_p, gmv_3d_p, expos_uv_3m_p, expos_uv_1m_p, expos_uv_15d_p, expos_uv_7d_p, expos_uv_3d_p, click_uv_3m_p, click_uv_1m_p, click_uv_15d_p, click_uv_7d_p, click_uv_3d_p, ctr_3m_p, ctr_1m_p, ctr_15d_p, ctr_7d_p, ctr_3d_p, unit_decay_gmv_3m_p, unit_decay_gmv_1m_p, unit_decay_gmv_15d_p, unit_decay_gmv_7d_p, unit_decay_gmv_3d_p, ctcvr_3m_p, ctcvr_1m_p, ctcvr_15d_p, ctcvr_7d_p, ctcvr_3d_p, cvr_3m_p, cvr_1m_p, cvr_15d_p, cvr_7d_p, cvr_3d_p, detail_uv_3m_p, detail_uv_1m_p, detail_uv_15d_p, detail_uv_7d_p, detail_uv_3d_p, order_cnt_3m_pl, order_cnt_1m_pl, order_cnt_15d_pl, order_cnt_7d_pl, order_cnt_3d_pl, gmv_3m_pl, gmv_1m_pl, gmv_15d_pl, gmv_7d_pl, gmv_3d_pl, expos_uv_3m_pl, expos_uv_1m_pl, expos_uv_15d_pl, expos_uv_7d_pl, expos_uv_3d_pl, click_uv_3m_pl, click_uv_1m_pl, click_uv_15d_pl, click_uv_7d_pl, click_uv_3d_pl, ctr_3m_pl, ctr_1m_pl, ctr_15d_pl, ctr_7d_pl, ctr_3d_pl, unit_decay_gmv_3m_pl, unit_decay_gmv_1m_pl, unit_decay_gmv_15d_pl, unit_decay_gmv_7d_pl, unit_decay_gmv_3d_pl, ctcvr_3m_pl, ctcvr_1m_pl, ctcvr_15d_pl, ctcvr_7d_pl, ctcvr_3d_pl, cvr_3m_pl, cvr_1m_pl, cvr_15d_pl, cvr_7d_pl, cvr_3d_pl, detail_uv_3m_pl, detail_uv_1m_pl, detail_uv_15d_pl, detail_uv_7d_pl, detail_uv_3d_pl, order_cnt_3m_pdk, order_cnt_1m_pdk, order_cnt_15d_pdk, order_cnt_7d_pdk, order_cnt_3d_pdk, gmv_3m_pdk, gmv_1m_pdk, gmv_15d_pdk, gmv_7d_pdk, gmv_3d_pdk, expos_uv_3m_pdk, expos_uv_1m_pdk, expos_uv_15d_pdk, expos_uv_7d_pdk, expos_uv_3d_pdk, click_uv_3m_pdk, click_uv_1m_pdk, click_uv_15d_pdk, click_uv_7d_pdk, click_uv_3d_pdk, ctr_3m_pdk, ctr_1m_pdk, ctr_15d_pdk, ctr_7d_pdk, ctr_3d_pdk, unit_decay_gmv_3m_pdk, unit_decay_gmv_1m_pdk, unit_decay_gmv_15d_pdk, unit_decay_gmv_7d_pdk, unit_decay_gmv_3d_pdk, ctcvr_3m_pdk, ctcvr_1m_pdk, ctcvr_15d_pdk, ctcvr_7d_pdk, ctcvr_3d_pdk, cvr_3m_pdk, cvr_1m_pdk, cvr_15d_pdk, cvr_7d_pdk, cvr_3d_pdk, order_cnt_3m_u, order_cnt_1m_u, order_cnt_15d_u, order_cnt_7d_u, order_cnt_3d_u, gmv_3m_u, gmv_1m_u, gmv_15d_u, gmv_7d_u, gmv_3d_u, expos_uv_3m_u, expos_uv_1m_u, expos_uv_15d_u, expos_uv_7d_u, expos_uv_3d_u, click_uv_3m_u, click_uv_1m_u, click_uv_15d_u, click_uv_7d_u, click_uv_3d_u, ctr_3m_u, ctr_1m_u, ctr_15d_u, ctr_7d_u, ctr_3d_u, unit_decay_gmv_3m_u, unit_decay_gmv_1m_u, unit_decay_gmv_15d_u, unit_decay_gmv_7d_u, unit_decay_gmv_3d_u, ctcvr_3m_u, ctcvr_1m_u, ctcvr_15d_u, ctcvr_7d_u, ctcvr_3d_u, cvr_3m_u, cvr_1m_u, cvr_15d_u, cvr_7d_u, cvr_3d_u, order_cnt_3m_uk, order_cnt_1m_uk, order_cnt_15d_uk, order_cnt_7d_uk, order_cnt_3d_uk, gmv_3m_uk, gmv_1m_uk, gmv_15d_uk, gmv_7d_uk, gmv_3d_uk, expos_uv_3m_uk, expos_uv_1m_uk, expos_uv_15d_uk, expos_uv_7d_uk, expos_uv_3d_uk, click_uv_3m_uk, click_uv_1m_uk, click_uv_15d_uk, click_uv_7d_uk, click_uv_3d_uk, ctr_3m_uk, ctr_1m_uk, ctr_15d_uk, ctr_7d_uk, ctr_3d_uk, unit_decay_gmv_3m_uk, unit_decay_gmv_1m_uk, unit_decay_gmv_15d_uk, unit_decay_gmv_7d_uk, unit_decay_gmv_3d_uk, ctcvr_3m_uk, ctcvr_1m_uk, ctcvr_15d_uk, ctcvr_7d_uk, ctcvr_3d_uk, cvr_3m_uk, cvr_1m_uk, cvr_15d_uk, cvr_7d_uk, cvr_3d_uk, order_cnt_3m_puk, order_cnt_1m_puk, order_cnt_15d_puk, order_cnt_7d_puk, order_cnt_3d_puk, gmv_3m_puk, gmv_1m_puk, gmv_15d_puk, gmv_7d_puk, gmv_3d_puk, expos_uv_3m_puk, expos_uv_1m_puk, expos_uv_15d_puk, expos_uv_7d_puk, expos_uv_3d_puk, click_uv_3m_puk, click_uv_1m_puk, click_uv_15d_puk, click_uv_7d_puk, click_uv_3d_puk, ctr_3m_puk, ctr_1m_puk, ctr_15d_puk, ctr_7d_puk, ctr_3d_puk, unit_decay_gmv_3m_puk, unit_decay_gmv_1m_puk, unit_decay_gmv_15d_puk, unit_decay_gmv_7d_puk, unit_decay_gmv_3d_puk, ctcvr_3m_puk, ctcvr_1m_puk, ctcvr_15d_puk, ctcvr_7d_puk, ctcvr_3d_puk, cvr_3m_puk, cvr_1m_puk, cvr_15d_puk, cvr_7d_puk, cvr_3d_puk, order_cnt_3m_k, order_cnt_1m_k, order_cnt_15d_k, order_cnt_7d_k, order_cnt_3d_k, gmv_3m_k, gmv_1m_k, gmv_15d_k, gmv_7d_k, gmv_3d_k, expos_uv_3m_k, expos_uv_1m_k, expos_uv_15d_k, expos_uv_7d_k, expos_uv_3d_k, click_uv_3m_k, click_uv_1m_k, click_uv_15d_k, click_uv_7d_k, click_uv_3d_k, ctr_3m_k, ctr_1m_k, ctr_15d_k, ctr_7d_k, ctr_3d_k, unit_decay_gmv_3m_k, unit_decay_gmv_1m_k, unit_decay_gmv_15d_k, unit_decay_gmv_7d_k, unit_decay_gmv_3d_k, ctcvr_3m_k, ctcvr_1m_k, ctcvr_15d_k, ctcvr_7d_k, ctcvr_3d_k, cvr_3m_k, cvr_1m_k, cvr_15d_k, cvr_7d_k, cvr_3d_k, order_cnt_3m_pu, order_cnt_1m_pu, order_cnt_15d_pu, order_cnt_7d_pu, order_cnt_3d_pu, gmv_3m_pu, gmv_1m_pu, gmv_15d_pu, gmv_7d_pu, gmv_3d_pu, expos_uv_3m_pu, expos_uv_1m_pu, expos_uv_15d_pu, expos_uv_7d_pu, expos_uv_3d_pu, click_uv_3m_pu, click_uv_1m_pu, click_uv_15d_pu, click_uv_7d_pu, click_uv_3d_pu, ctr_3m_pu, ctr_1m_pu, ctr_15d_pu, ctr_7d_pu, ctr_3d_pu, unit_decay_gmv_3m_pu, unit_decay_gmv_1m_pu, unit_decay_gmv_15d_pu, unit_decay_gmv_7d_pu, unit_decay_gmv_3d_pu, ctcvr_3m_pu, ctcvr_1m_pu, ctcvr_15d_pu, ctcvr_7d_pu, ctcvr_3d_pu, cvr_3m_pu, cvr_1m_pu, cvr_15d_pu, cvr_7d_pu, cvr_3d_pu'.split(',')]
# deep_sparse_features = [x.strip() for x in 'districtid_enc_index, user_residgscityid_enc_index, localdistrictid_enc_index, userlocaltype_enc_index, district_userlocaltype_enc_index, age_enc_index, gender_enc_index, star_enc_index, member_level_enc_index, crown_enc_index, user_value_enc_index, themeid_enc_index'.split(',')]

In [6]:
# holiday, 长时间周期暂时不加
# 只用15d统计特征

# wide_dense
wide_dense_str = 'around_tour,isholiday,hot_p1m,ctr_p1m,order_p1m,gmv_p1m,unitgmv_p1m,comment_p,comment_pl2m,hot_pla1m,\
ctr_pla1m,order_pla1m,gmv_pla1m,unitgmv_pla1m,queryctr_pldk1m,new_p1m,season_p,basic_p,kwdcoeff_p,text_score,\
businessstatus,isheat,imquality_score,ishighqualitypoi,isfree'

# wide_sparse
wide_sparse_str = 'date_type'

# deep_dense
deep_dense_str = 'around_tour,isholiday,hot_p1m,ctr_p1m,order_p1m,gmv_p1m,unitgmv_p1m,comment_p,comment_pl2m,hot_pla1m,\
ctr_pla1m,order_pla1m,gmv_pla1m,unitgmv_pla1m,queryctr_pldk1m,new_p1m,season_p,basic_p,kwdcoeff_p,text_score,\
businessstatus,isheat,imquality_score,ishighqualitypoi,isfree,\
order_cnt_15d_p,gmv_15d_p,expos_uv_15d_p,click_uv_15d_p,ctr_15d_p,unit_decay_gmv_15d_p,ctcvr_15d_p,cvr_15d_p,\
detail_uv_15d_p,order_cnt_15d_pl,gmv_15d_pl,expos_uv_15d_pl,click_uv_15d_pl,ctr_15d_pl,unit_decay_gmv_15d_pl,\
ctcvr_15d_pl,cvr_15d_pl,detail_uv_15d_pl,order_cnt_15d_pdk,gmv_15d_pdk,expos_uv_15d_pdk,click_uv_15d_pdk,ctr_15d_pdk,\
unit_decay_gmv_15d_pdk,ctcvr_15d_pdk,cvr_15d_pdk,order_cnt_15d_u,gmv_15d_u,expos_uv_15d_u,click_uv_15d_u,ctr_15d_u,\
unit_decay_gmv_15d_u,ctcvr_15d_u,cvr_15d_u,order_cnt_15d_uk,gmv_15d_uk,expos_uv_15d_uk,click_uv_15d_uk,ctr_15d_uk,\
unit_decay_gmv_15d_uk,ctcvr_15d_uk,cvr_15d_uk,order_cnt_15d_puk,gmv_15d_puk,expos_uv_15d_puk,click_uv_15d_puk,ctr_15d_puk,\
unit_decay_gmv_15d_puk,ctcvr_15d_puk,cvr_15d_puk,order_cnt_15d_k,gmv_15d_k,expos_uv_15d_k,click_uv_15d_k,ctr_15d_k,\
unit_decay_gmv_15d_k,ctcvr_15d_k,cvr_15d_k,order_cnt_15d_pu,gmv_15d_pu,expos_uv_15d_pu,click_uv_15d_pu,ctr_15d_pu,\
unit_decay_gmv_15d_pu,ctcvr_15d_pu,cvr_15d_pu'

# deep_sparse
deep_sparse_str = 'date_type,\
userlocaltype_enc_index,district_userlocaltype_enc_index,age_enc_index,gender_enc_index,\
star_enc_index,member_level_enc_index,crown_enc_index,user_value_enc_index'

# 序列id特征
behavior_feature_str = 'viewspotid_enc_index'



In [7]:
# 只取前100,000的景点id
viewspotid_enc_index_max = 100000
sample = -1
train_file = '../dataset/tkt_kwd_train_707_727.csv'
data = pd.read_csv(train_file)
data['viewspotid_enc_index'] = np.where(data['viewspotid_enc_index'] <= viewspotid_enc_index_max, data['viewspotid_enc_index'], 0)
print('train samples:', len(data))

/opt/app/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (50) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


train samples: 259584


In [8]:
# features
wide_dense_features = [x.strip() for x in wide_dense_str.split(',')]
wide_sparse_features = [x.strip() for x in wide_sparse_str.split(',')]
deep_dense_features = [x.strip() for x in deep_dense_str.split(',')]
deep_sparse_features = [x.strip() for x in deep_sparse_str.split(',')]
# 所有dense和sparse
dense_features = list(set(wide_dense_features + deep_dense_features))
sparse_features = list(set(wide_sparse_features + deep_sparse_features))
sparse_features_num = dict(data[sparse_features].max())
print('连续特征个数: ', len(dense_features))
print('离散特征个数: ', len(sparse_features))
print('wide部分特征个数: ', len(wide_dense_features) + len(wide_sparse_features))
print('deep部分特征个数: ', len(deep_dense_features) + len(deep_sparse_features))

# 景点id特征
behavior_feature = [x.strip() for x in behavior_feature_str.split(',')]

连续特征个数:  91
离散特征个数:  9
wide部分特征个数:  26
deep部分特征个数:  100


In [9]:
embed_dim = 16
seq_len = 20
dense_feature_list = [denseFeature(feat) for feat in dense_features] # 所有连续特征
sparse_feature_list = [sparseFeature(feat, feat_num=sparse_features_num[feat], embed_dim=embed_dim) for feat in sparse_features] # 所有离散特征
all_feature_list = dense_feature_list + sparse_feature_list # 所有特征list
# 分别指定wide  deep侧特征
wide_feature_list = [denseFeature(feat) for feat in wide_dense_features] + \
                    [sparseFeature(feat, feat_num=sparse_features_num[feat], embed_dim=embed_dim) for feat in wide_sparse_features]# 指定wide侧的特征，不指定则默认所有
deep_feature_list = [denseFeature(feat) for feat in deep_dense_features] + \
                    [sparseFeature(feat, feat_num=sparse_features_num[feat], embed_dim=embed_dim) for feat in deep_sparse_features]# 指定deep侧的特征，不指定则默认所有
print('wide部分特征个数: ', len(wide_feature_list))
print('deep部分特征个数: ', len(deep_feature_list))
# 景点id特征
behavior_feature_list = [sparseFeature(feat, feat_num=viewspotid_enc_index_max, embed_dim=embed_dim) for feat in behavior_feature]

wide部分特征个数:  26
deep部分特征个数:  100


In [10]:
behavior_feature_list

[{'feat_name': 'viewspotid_enc_index',
  'feat_num': 100000,
  'embed_dim': 16,
  'type': 'sparse'}]

## model

In [11]:
model = WideDeepDIN(dense_feature_list, sparse_feature_list, wide_feature_list, deep_feature_list,
                hidden_units=[128,256,64], activation='relu',
                behavior_feature_list=behavior_feature_list, att_hidden_units=[64,32], att_activation='relu', seq_len=seq_len,
                dnn_dropout=0.1, embed_reg=1e-4, w_reg=1e-3, att_w_reg=1e-5, att_softmax=True) 
model.summary()

wide side featrues:  around_tour,isholiday,hot_p1m,ctr_p1m,order_p1m,gmv_p1m,unitgmv_p1m,comment_p,comment_pl2m,hot_pla1m,ctr_pla1m,order_pla1m,gmv_pla1m,unitgmv_pla1m,queryctr_pldk1m,new_p1m,season_p,basic_p,kwdcoeff_p,text_score,businessstatus,isheat,imquality_score,ishighqualitypoi,isfree,date_type
deep side featrues:  around_tour,isholiday,hot_p1m,ctr_p1m,order_p1m,gmv_p1m,unitgmv_p1m,comment_p,comment_pl2m,hot_pla1m,ctr_pla1m,order_pla1m,gmv_pla1m,unitgmv_pla1m,queryctr_pldk1m,new_p1m,season_p,basic_p,kwdcoeff_p,text_score,businessstatus,isheat,imquality_score,ishighqualitypoi,isfree,order_cnt_15d_p,gmv_15d_p,expos_uv_15d_p,click_uv_15d_p,ctr_15d_p,unit_decay_gmv_15d_p,ctcvr_15d_p,cvr_15d_p,detail_uv_15d_p,order_cnt_15d_pl,gmv_15d_pl,expos_uv_15d_pl,click_uv_15d_pl,ctr_15d_pl,unit_decay_gmv_15d_pl,ctcvr_15d_pl,cvr_15d_pl,detail_uv_15d_pl,order_cnt_15d_pdk,gmv_15d_pdk,expos_uv_15d_pdk,click_uv_15d_pdk,ctr_15d_pdk,unit_decay_gmv_15d_pdk,ctcvr_15d_pdk,cvr_15d_pdk,order_cnt_15d_u,gmv_

In [12]:
# model.attention_layer

In [12]:
# 点击 pkgid_enc_index_click_pkgid_list_u
# 下单 pkgid_enc_index_order_pkgid_list_u

train_X, train_y, seq_input, item_input = get_model_input(data, all_feature_list=all_feature_list, target='isorder', over_sample_rate=3, task='train',
                    item_col='viewspotid_enc_index', seq_col='pkgid_enc_index_order_pkgid_list_u', 
                    seq_len=seq_len, item_enc_max=viewspotid_enc_index_max)

In [13]:
print('正样本个数: {}, 负样本个数: {}, 总样本数: {}'.format(train_y.sum(), len(train_y) - train_y.sum(), len(train_y)))
learning_rate = 0.001
batch_size = 1024
epochs = 50
model.compile(loss=binary_crossentropy, optimizer=Adam(learning_rate=learning_rate),
                      metrics=[AUC()]) # BinaryAccuracy(threshold=0.5)

正样本个数: 135444, 负样本个数: 214436, 总样本数: 349880


In [14]:
model.fit(
    [train_X, seq_input, item_input],
    train_y,
    epochs=epochs,
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],  # checkpoint
    batch_size=batch_size,
    validation_split=0.1
)

Epoch 1/50
308/308 [==============================] - 13s 26ms/step - loss: 0.4057 - auc: 0.9504 - val_loss: 0.2827 - val_auc: 0.9658
Epoch 2/50
308/308 [==============================] - 7s 22ms/step - loss: 0.2721 - auc: 0.9669 - val_loss: 0.2604 - val_auc: 0.9692
Epoch 3/50
308/308 [==============================] - 7s 21ms/step - loss: 0.2585 - auc: 0.9692 - val_loss: 0.2545 - val_auc: 0.9699
Epoch 4/50
308/308 [==============================] - 6s 21ms/step - loss: 0.2531 - auc: 0.9703 - val_loss: 0.2526 - val_auc: 0.9704
Epoch 5/50
308/308 [==============================] - 7s 21ms/step - loss: 0.2510 - auc: 0.9707 - val_loss: 0.2502 - val_auc: 0.9709
Epoch 6/50
308/308 [==============================] - 7s 21ms/step - loss: 0.2491 - auc: 0.9711 - val_loss: 0.2518 - val_auc: 0.9713
Epoch 7/50
308/308 [==============================] - 6s 21ms/step - loss: 0.2474 - auc: 0.9717 - val_loss: 0.2466 - val_auc: 0.9721
Epoch 8/50
308/308 [==============================] - 7s 22ms/step -

## 测试集

In [25]:
data['poi_district_distance'].describe()

count    2.595840e+05
mean     3.763547e-01
std      4.103151e-01
min      3.200084e-07
25%      1.227214e-02
50%      1.444384e-01
75%      8.911883e-01
max      1.000000e+00
Name: poi_district_distance, dtype: float64

In [15]:
# test_file = '../dataset/tkt_kwd_test_728_730.csv'
# test_file = '../dataset/tkt_kwd_test_728_730_vid.csv'
test_file = '../dataset/tkt_kwd_test_728_730_order.csv'
test_data = pd.read_csv(test_file)
test_data['viewspotid_enc_index'] = np.where(test_data['viewspotid_enc_index'] <= viewspotid_enc_index_max, test_data['viewspotid_enc_index'], 0)
print('test samples:', len(test_data))

test samples: 7301


In [ ]:
# 点击 pkgid_enc_index_click_pkgid_list_u
# 下单 pkgid_enc_index_order_pkgid_list_u

In [16]:
test_X, test_y, test_seq_input, test_item_input = get_model_input(test_data, all_feature_list=all_feature_list, 
                                                                  target='isorder', over_sample_rate=-1, task='test',
                    item_col='viewspotid_enc_index', seq_col='pkgid_enc_index_order_pkgid_list_u', 
                    seq_len=seq_len, item_enc_max=viewspotid_enc_index_max)

In [24]:
# test_X, test_y = get_model_input(test_data, all_feature_list=all_feature_list, target='isorder', task='test')
# loss = model.evaluate([test_X, test_seq_input, test_item_input], test_y, batch_size=batch_size)
# print('test loss: %f' % loss[0])
# print('test AUC: %f' % loss[1])
# # print('test accuracy: %f' % loss[2])

In [26]:
# df_metric = test_data[['traceid', 'fine_rank', 'isorder']].copy()
# df_metric['pred_score'] = 1 / df_metric['fine_rank']

# gauc = get_gauc(df_metric,qid_col='traceid',label_col='isorder',y_pred_col='pred_score')
# print('gauc: {}'.format(np.round(gauc, 4)))
# n_list = [1,3,5,10,20,30]
# for n in n_list:
#     m = get_topn(df_metric,label_col='isorder',rank_col='fine_rank',n=n)
#     print('top{}: {}'.format(n, np.round(m, 4)))

In [17]:
pred = model.predict([test_X, test_seq_input, test_item_input], batch_size=batch_size)
df_metric = test_data[['traceid', 'fine_rank', 'isorder']].copy()
df_metric['pred_score'] = 1 / df_metric['fine_rank']
df_metric['pred_score_deep_model'] = pred
df_metric['fine_rank_deep_model'] = df_metric.groupby(['traceid'])['pred_score_deep_model'].rank('dense', ascending=False).astype('int32')

In [18]:
gauc = get_gauc(df_metric,qid_col='traceid',label_col='isorder',y_pred_col='pred_score_deep_model')
print('gauc: {}'.format(np.round(gauc, 4)))
n_list = [1,3,5,10,20,30]
for n in n_list:
    m = get_topn(df_metric,label_col='isorder',rank_col='fine_rank_deep_model',n=n)
    print('top{}: {}'.format(n, np.round(m, 4)))

gauc: 0.8979
top1: 0.823
top3: 0.942
top5: 0.9733
top10: 0.9939
top20: 0.9992
top30: 1.0


In [35]:
# test_data[(test_data['pkgid_enc_index_order_pkgid_list_u'].notna()) & (test_data['viewspotid_enc_index'] != 0)].to_csv('../dataset/tkt_kwd_test_728_730_order.csv', index=False)

In [29]:
df_metric[df_metric['isorder'] == 1].sort_values(['pred_score_deep_model'])[:30]

,traceid,fine_rank,isorder,pred_score_deep_model,fine_rank_deep_model
3719,d69f9424-220c-25dc-78c7-1659065bf844,6,1,1.686742e-07,8
21678,5e655935-90ae-e307-7454-109436228660,4,1,1.847608e-07,11
18667,fe71ff33-6d08-f182-1e21-165907fdb303,5,1,2.869936e-07,6
18708,7f905628-b9f8-0825-8070-16fd03211967,5,1,2.869936e-07,6
2250,0ae04e5d-c21c-ac57-0a04-d94777655126,1,1,4.922885e-07,7
35857,87d50d68-416b-ca40-6e67-129d10096777,5,1,1.881201e-06,7
21947,1d4c9784-f327-c35e-a893-1639b4827862,1,1,4.121163e-06,3
34759,7f3de4fa-8d4b-69c5-0037-16596f324196,5,1,4.337428e-06,28
45960,036c7dae-d59e-b4e5-9932-165898273eb3,1,1,6.612051e-06,3
24,18be31ed-19a7-08c0-7b6e-1832a9405070,2,1,1.271807e-05,2


In [59]:
model.save('saved_model/widedeep_v1_vid_att')

INFO:tensorflow:Assets written to: saved_model/widedeep_v1_vid_att/assets


INFO:tensorflow:Assets written to: saved_model/widedeep_v1_vid_att/assets


In [49]:
model.save_weights('./checkpoints/widedeep_v1_6432_relu_dim8_drop02_lr0001_epoch15')

In [51]:
model.save('saved_model/widedeep_v1_6432_relu_dim8_drop02_lr0001_epoch15')

INFO:tensorflow:Assets written to: saved_model/widedeep_v1_6432_relu_dim8_drop02_lr0001_epoch15/assets


INFO:tensorflow:Assets written to: saved_model/widedeep_v1_6432_relu_dim8_drop02_lr0001_epoch15/assets


In [6]:
# # example
# # 两个 dense feature
# dense_f1 = denseFeature('dense_feature_1')
# dense_f2 = denseFeature('dense_feature_2')
# # 两个 sparse feature
# sparse_f1 = sparseFeature('sparse_feature_1', feat_num=2, embed_dim=8) 
# sparse_f2 = sparseFeature('sparse_feature_2', feat_num=5, embed_dim=8)
# # 
# dense_feature_list = [dense_f1, dense_f2] # 所有连续特征
# sparse_feature_list = [sparse_f1, sparse_f2] # 所有离散特征
# # 分别指定wide  deep侧特征
# wide_feature_list = [dense_f1, dense_f2] # 指定wide侧的特征，不指定则默认所有
# deep_feature_list = [dense_f2, sparse_f1] # 指定deep侧的特征，不指定则默认所有


# model = WideDeep(dense_feature_list, sparse_feature_list, wide_feature_list, deep_feature_list) 
# model.summary()

# batch = 128
# d_f1 = tf.constant(np.random.rand(batch, 1))
# d_f2 = tf.constant(np.random.rand(batch, 1))
# s_f1 = tf.constant(np.random.randint(2, size=(batch, 1)), dtype=tf.float32)
# s_f2 = tf.constant(np.random.randint(5, size=(batch, 1)), dtype=tf.float32)
# inputs = [d_f1, d_f2, s_f1, s_f2]
# outputs = model(inputs)
# print(outputs.shape)

In [63]:
# features = pd.DataFrame({'feature': wide_dense + deep_dense + sparse, 
#               'side': ['wide'] * len(wide_dense) + ['deep'] * len(deep_dense) + ['sparse'] * len(sparse)})

# pd.merge(features, f_info, how='left', on=['feature']).to_csv('../dataset/model_feature_info.csv', index=False, encoding='utf-8')

In [21]:
del data,  test_data

In [27]:
sql = "SELECT traceid, fine_rank,\
    viewspotid_enc_index, districtid_enc_index, user_residgscityid_enc_index, localdistrictid_enc_index, userlocaltype_enc_index\
    , around_tour, isholiday, isclick, isorder, is_unexposed_product\
    , district_userlocaltype_enc_index, age_enc_index, gender_enc_index, star_enc_index, member_level_enc_index\
    , crown_enc_index, user_value_enc_index, hot_p1m, ctr_p1m, order_p1m\
    , gmv_p1m, unitgmv_p1m, comment_p, comment_pl2m, hot_pla1m\
    , ctr_pla1m, order_pla1m, gmv_pla1m, unitgmv_pla1m, themeid_enc_index\
    , queryctr_pldk1m, new_p1m, season_p, basic_p, kwdcoeff_p\
    , text_score, local_poi_distance, local_district_distance, poi_district_distance, businessstatus\
    , dat, date_type, holiday_type, day_of_week, week_of_year\
    , month, pkgid_enc_index_click_pkgid_list_uk, click_seq_mask_uk, pkgid_enc_index_order_pkgid_list_uk, order_seq_mask_uk\
    , pkgid_enc_index_click_pkgid_list_u, click_seq_mask_u, pkgid_enc_index_order_pkgid_list_u, order_seq_mask_u, order_cnt_3m_p\
    , order_cnt_1m_p, order_cnt_15d_p, order_cnt_7d_p, order_cnt_3d_p, gmv_3m_p\
    , gmv_1m_p, gmv_15d_p, gmv_7d_p, gmv_3d_p, expos_uv_3m_p\
    , expos_uv_1m_p, expos_uv_15d_p, expos_uv_7d_p, expos_uv_3d_p, click_uv_3m_p\
    , click_uv_1m_p, click_uv_15d_p, click_uv_7d_p, click_uv_3d_p, ctr_3m_p\
    , ctr_1m_p, ctr_15d_p, ctr_7d_p, ctr_3d_p, unit_decay_gmv_3m_p\
    , unit_decay_gmv_1m_p, unit_decay_gmv_15d_p, unit_decay_gmv_7d_p, unit_decay_gmv_3d_p, ctcvr_3m_p\
    , ctcvr_1m_p, ctcvr_15d_p, ctcvr_7d_p, ctcvr_3d_p, cvr_3m_p\
    , cvr_1m_p, cvr_15d_p, cvr_7d_p, cvr_3d_p, detail_uv_3m_p\
    , detail_uv_1m_p, detail_uv_15d_p, detail_uv_7d_p, detail_uv_3d_p, isheat\
    , imquality_score, ishighqualitypoi, iscoverimage, hasticket, inchina\
    , isfree, order_cnt_3m_pl, order_cnt_1m_pl, order_cnt_15d_pl, order_cnt_7d_pl\
    , order_cnt_3d_pl, gmv_3m_pl, gmv_1m_pl, gmv_15d_pl, gmv_7d_pl\
    , gmv_3d_pl, expos_uv_3m_pl, expos_uv_1m_pl, expos_uv_15d_pl, expos_uv_7d_pl\
    , expos_uv_3d_pl, click_uv_3m_pl, click_uv_1m_pl, click_uv_15d_pl, click_uv_7d_pl\
    , click_uv_3d_pl, ctr_3m_pl, ctr_1m_pl, ctr_15d_pl, ctr_7d_pl\
    , ctr_3d_pl, unit_decay_gmv_3m_pl, unit_decay_gmv_1m_pl, unit_decay_gmv_15d_pl, unit_decay_gmv_7d_pl\
    , unit_decay_gmv_3d_pl, ctcvr_3m_pl, ctcvr_1m_pl, ctcvr_15d_pl, ctcvr_7d_pl\
    , ctcvr_3d_pl, cvr_3m_pl, cvr_1m_pl, cvr_15d_pl, cvr_7d_pl\
    , cvr_3d_pl, detail_uv_3m_pl, detail_uv_1m_pl, detail_uv_15d_pl, detail_uv_7d_pl\
    , detail_uv_3d_pl, order_cnt_3m_pdk, order_cnt_1m_pdk, order_cnt_15d_pdk, order_cnt_7d_pdk\
    , order_cnt_3d_pdk, gmv_3m_pdk, gmv_1m_pdk, gmv_15d_pdk, gmv_7d_pdk\
    , gmv_3d_pdk, expos_uv_3m_pdk, expos_uv_1m_pdk, expos_uv_15d_pdk, expos_uv_7d_pdk\
    , expos_uv_3d_pdk, click_uv_3m_pdk, click_uv_1m_pdk, click_uv_15d_pdk, click_uv_7d_pdk\
    , click_uv_3d_pdk, ctr_3m_pdk, ctr_1m_pdk, ctr_15d_pdk, ctr_7d_pdk\
    , ctr_3d_pdk, unit_decay_gmv_3m_pdk, unit_decay_gmv_1m_pdk, unit_decay_gmv_15d_pdk, unit_decay_gmv_7d_pdk\
    , unit_decay_gmv_3d_pdk, ctcvr_3m_pdk, ctcvr_1m_pdk, ctcvr_15d_pdk, ctcvr_7d_pdk\
    , ctcvr_3d_pdk, cvr_3m_pdk, cvr_1m_pdk, cvr_15d_pdk, cvr_7d_pdk\
    , cvr_3d_pdk, order_cnt_3m_u, order_cnt_1m_u, order_cnt_15d_u, order_cnt_7d_u\
    , order_cnt_3d_u, gmv_3m_u, gmv_1m_u, gmv_15d_u, gmv_7d_u\
    , gmv_3d_u, expos_uv_3m_u, expos_uv_1m_u, expos_uv_15d_u, expos_uv_7d_u\
    , expos_uv_3d_u, click_uv_3m_u, click_uv_1m_u, click_uv_15d_u, click_uv_7d_u\
    , click_uv_3d_u, ctr_3m_u, ctr_1m_u, ctr_15d_u, ctr_7d_u\
    , ctr_3d_u, unit_decay_gmv_3m_u, unit_decay_gmv_1m_u, unit_decay_gmv_15d_u, unit_decay_gmv_7d_u\
    , unit_decay_gmv_3d_u, ctcvr_3m_u, ctcvr_1m_u, ctcvr_15d_u, ctcvr_7d_u\
    , ctcvr_3d_u, cvr_3m_u, cvr_1m_u, cvr_15d_u, cvr_7d_u\
    , cvr_3d_u, order_cnt_3m_uk, order_cnt_1m_uk, order_cnt_15d_uk, order_cnt_7d_uk\
    , order_cnt_3d_uk, gmv_3m_uk, gmv_1m_uk, gmv_15d_uk, gmv_7d_uk\
    , gmv_3d_uk, expos_uv_3m_uk, expos_uv_1m_uk, expos_uv_15d_uk, expos_uv_7d_uk\
    , expos_uv_3d_uk, click_uv_3m_uk, click_uv_1m_uk, click_uv_15d_uk, click_uv_7d_uk\
    , click_uv_3d_uk, ctr_3m_uk, ctr_1m_uk, ctr_15d_uk, ctr_7d_uk\
    , ctr_3d_uk, unit_decay_gmv_3m_uk, unit_decay_gmv_1m_uk, unit_decay_gmv_15d_uk, unit_decay_gmv_7d_uk\
    , unit_decay_gmv_3d_uk, ctcvr_3m_uk, ctcvr_1m_uk, ctcvr_15d_uk, ctcvr_7d_uk\
    , ctcvr_3d_uk, cvr_3m_uk, cvr_1m_uk, cvr_15d_uk, cvr_7d_uk\
    , cvr_3d_uk, order_cnt_3m_puk, order_cnt_1m_puk, order_cnt_15d_puk, order_cnt_7d_puk\
    , order_cnt_3d_puk, gmv_3m_puk, gmv_1m_puk, gmv_15d_puk, gmv_7d_puk\
    , gmv_3d_puk, expos_uv_3m_puk, expos_uv_1m_puk, expos_uv_15d_puk, expos_uv_7d_puk\
    , expos_uv_3d_puk, click_uv_3m_puk, click_uv_1m_puk, click_uv_15d_puk, click_uv_7d_puk\
    , click_uv_3d_puk, ctr_3m_puk, ctr_1m_puk, ctr_15d_puk, ctr_7d_puk\
    , ctr_3d_puk, unit_decay_gmv_3m_puk, unit_decay_gmv_1m_puk, unit_decay_gmv_15d_puk, unit_decay_gmv_7d_puk\
    , unit_decay_gmv_3d_puk, ctcvr_3m_puk, ctcvr_1m_puk, ctcvr_15d_puk, ctcvr_7d_puk\
    , ctcvr_3d_puk, cvr_3m_puk, cvr_1m_puk, cvr_15d_puk, cvr_7d_puk\
    , cvr_3d_puk, order_cnt_3m_k, order_cnt_1m_k, order_cnt_15d_k, order_cnt_7d_k\
    , order_cnt_3d_k, gmv_3m_k, gmv_1m_k, gmv_15d_k, gmv_7d_k\
    , gmv_3d_k, expos_uv_3m_k, expos_uv_1m_k, expos_uv_15d_k, expos_uv_7d_k\
    , expos_uv_3d_k, click_uv_3m_k, click_uv_1m_k, click_uv_15d_k, click_uv_7d_k\
    , click_uv_3d_k, ctr_3m_k, ctr_1m_k, ctr_15d_k, ctr_7d_k\
    , ctr_3d_k, unit_decay_gmv_3m_k, unit_decay_gmv_1m_k, unit_decay_gmv_15d_k, unit_decay_gmv_7d_k\
    , unit_decay_gmv_3d_k, ctcvr_3m_k, ctcvr_1m_k, ctcvr_15d_k, ctcvr_7d_k\
    , ctcvr_3d_k, cvr_3m_k, cvr_1m_k, cvr_15d_k, cvr_7d_k\
    , cvr_3d_k, order_cnt_3m_pu, order_cnt_1m_pu, order_cnt_15d_pu, order_cnt_7d_pu\
    , order_cnt_3d_pu, gmv_3m_pu, gmv_1m_pu, gmv_15d_pu, gmv_7d_pu\
    , gmv_3d_pu, expos_uv_3m_pu, expos_uv_1m_pu, expos_uv_15d_pu, expos_uv_7d_pu\
    , expos_uv_3d_pu, click_uv_3m_pu, click_uv_1m_pu, click_uv_15d_pu, click_uv_7d_pu\
    , click_uv_3d_pu, ctr_3m_pu, ctr_1m_pu, ctr_15d_pu, ctr_7d_pu\
    , ctr_3d_pu, unit_decay_gmv_3m_pu, unit_decay_gmv_1m_pu, unit_decay_gmv_15d_pu, unit_decay_gmv_7d_pu\
    , unit_decay_gmv_3d_pu, ctcvr_3m_pu, ctcvr_1m_pu, ctcvr_15d_pu, ctcvr_7d_pu\
    , ctcvr_3d_pu, cvr_3m_pu, cvr_1m_pu, cvr_15d_pu, cvr_7d_pu\
    , cvr_3d_pu \
FROM ods_actttdsearchdb.adm_srh_algo_ctrip_tkt_blackbox_ranking_sample_detail \
WHERE d BETWEEN '2022-07-07' AND '2022-07-27' \
    AND is_unexposed_product = 0 \
    AND order_in_trace > 0"

In [ ]:
spark_df = spark.sql(sql)
data = spark_df.toPandas()

In [ ]:
data.shape

In [12]:
data.to_csv('../dataset/tkt_kwd_test_728_730.csv', index=False)

In [ ]:
tkt_kwd_train_707_727